# Training

# Import packages

In [62]:
import pandas as pd
from pathlib import Path
import numpy as np
import ast

from lightning import pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint

from chemprop import data, featurizers, models, nn
from chemprop.utils import make_mol

# Change data inputs here

In [63]:
chemprop_dir = Path.cwd().parent
input_path = chemprop_dir / "tests" / "data" / "mixed_regression_input.csv" # path to your data .csv file
num_workers = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'smiles' # name of the column containing SMILES strings
target_columns = ['molecule', 'atom', 'bond'] # list of names of the columns containing targets

In [64]:
df_input = pd.read_csv(input_path)
df_input

,smiles,molecule,atom,bond
0,CC,1.0,"[1,2]",[1]
1,CCC,2.0,"[1,2,3]","[1,2]"
2,CCCO,3.0,"[1,2,3,5]","[1,2,3]"
3,CCOO,4.0,"[1,2,3,4]","[1,2,3]"
4,COO,5.0,"[1,3,5]","[1,2]"
5,CCOOO,6.0,"[1,7,3,4,5]","[1,3,2,4]"
6,COOO,7.0,"[2,5,3,2]","[2,3,4]"
7,CO,8.0,"[1,3]",[2]
8,CCO,9.0,"[1,3,5]","[2,3]"
9,OO,10.0,"[5,7]",[3]


## Get SMILES and targets

In [65]:
smis = df_input.loc[:, smiles_column].values
ys = df_input.loc[:, target_columns]

In [66]:
smis[:2] # show first 2 SMILES strings

array(['CC', 'CCC'], dtype=object)

In [67]:
ys[:5] # show first 5 molecule targets

,molecule,atom,bond
0,1.0,"[1,2]",[1]
1,2.0,"[1,2,3]","[1,2]"
2,3.0,"[1,2,3,5]","[1,2,3]"
3,4.0,"[1,2,3,4]","[1,2,3]"
4,5.0,"[1,3,5]","[1,2]"


In [68]:
import numpy as np

flag, mol_Y, atom_Y, bond_Y = [], [], [], []
for column in target_columns:
    index = 0
    column_type = df_input.iloc[index][column]
    if isinstance(column_type, float):
        for molecule in range(len(df_input)):
            mol_Y.append([df_input.iloc[molecule][column]])
        flag.append("mol")
    else:
        column_mol = make_mol(df_input.iloc[index][smiles_column], False, False)
        column_type = ast.literal_eval(column_type)
        while index < len(df_input) and column_mol.GetNumAtoms() == column_mol.GetNumBonds():
            index += 1
            column_mol = make_mol(df_input.iloc[index][smiles_column], False, False)
        column_type = ast.literal_eval(df_input.iloc[index][column])
        flag.append("atom") if len(column_type) == column_mol.GetNumAtoms() else flag.append("bond")
        
for molecule in range(len(df_input)):
    atom_list_props = []
    bond_list_props = []
    for prop in range(len(target_columns)):
        if flag[prop] == "mol":
            continue
        np_prop = np.array(ast.literal_eval(df_input.iloc[molecule][target_columns[prop]]))
        np_prop = np.expand_dims(np_prop, axis=1)
        atom_list_props.append(np_prop) if flag[prop] == "atom" else bond_list_props.append(np_prop)
    if len(atom_list_props) > 0:
        atom_Y.append(np.hstack(atom_list_props))
    if len(bond_list_props) > 0:
        bond_Y.append(np.hstack(bond_list_props))

print(mol_Y)

[[1.0], [2.0], [3.0], [4.0], [5.0], [6.0], [7.0], [8.0], [9.0], [10.0]]


## Get molecule datapoints

In [69]:
all_data = []

mol_data = []
for smi, y in zip(smis, mol_Y):
    mol_data.append(data.MoleculeDatapoint.from_smi(smi, y, keep_h=True))

atom_data = []
for smi, y in zip(smis, atom_Y):
    atom_data.append(data.MoleculeDatapoint.from_smi(smi, y, keep_h=True))

bond_data = []
for smi, y in zip(smis, bond_Y):
    bond_data.append(data.MoleculeDatapoint.from_smi(smi, y, keep_h=True))

all_data.append(mol_data)
all_data.append(atom_data)
all_data.append(bond_data)

## Perform data splitting for training, validation, and testing

In [70]:
# available split types
list(data.SplitType.keys())

['SCAFFOLD_BALANCED',
 'RANDOM_WITH_REPEATED_SMILES',
 'RANDOM',
 'KENNARD_STONE',
 'KMEANS']

In [71]:
mol = [d.mol for d in all_data[0]]

train_indices, val_indices, test_indices = data.make_split_indices(mol, "random", (0.6, 0.2, 0.2))
print(all_data)
train_data, val_data, test_data = data.split_data_by_indices(
    all_data, train_indices, val_indices, test_indices
)


[[MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x177104740>, y=[1.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='CC', V_f=None, E_f=None, V_d=None, E_d=None), MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x1773ae9d0>, y=[2.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='CCC', V_f=None, E_f=None, V_d=None, E_d=None), MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x1773aeea0>, y=[3.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='CCCO', V_f=None, E_f=None, V_d=None, E_d=None), MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x1773aef10>, y=[4.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='CCOO', V_f=None, E_f=None, V_d=None, E_d=None), MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x1773af220>, y=[5.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='COO', V_f=None, E_f=None, V_d=None, E_d=None), MoleculeDatapoint(mol=

## Get AtomDataset

In [72]:
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

train_dsets = []
train_dsets.append(data.MoleculeDataset(train_data[0][0], featurizer))
train_dsets.append(data.AtomDataset(train_data[0][1], featurizer))
train_dsets.append(data.BondDataset(train_data[0][2], featurizer))
print(train_dsets[0])
mol_scaler = train_dsets[0].normalize_targets()
atom_scaler = train_dsets[1].normalize_targets()
bond_scaler = train_dsets[2].normalize_targets()
train_dset = data.MolAtomBondDataset(train_dsets[0], train_dsets[1], train_dsets[2])

MoleculeDataset(data=[MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x1773af5a0>, y=[9.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='CCO', V_f=None, E_f=None, V_d=None, E_d=None), MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x1773af220>, y=[5.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='COO', V_f=None, E_f=None, V_d=None, E_d=None), MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x1773af680>, y=[10.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='OO', V_f=None, E_f=None, V_d=None, E_d=None), MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x1773ae9d0>, y=[2.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='CCC', V_f=None, E_f=None, V_d=None, E_d=None), MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x1773af3e0>, y=[7.0], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='COOO', V_f=None, E_f=None, V_d=None, E_d=None), Mo

In [73]:
val_dsets = []
val_dsets.append(data.MoleculeDataset(val_data[0][0], featurizer))
val_dsets.append(data.AtomDataset(val_data[0][1], featurizer))
val_dsets.append(data.BondDataset(val_data[0][2], featurizer))
val_dsets[0].normalize_targets(mol_scaler)
val_dsets[1].normalize_targets(atom_scaler)
val_dsets[2].normalize_targets(bond_scaler)
val_dset = data.MolAtomBondDataset(val_dsets[0], val_dsets[1], val_dsets[2])

test_dsets = []
test_dsets.append(data.MoleculeDataset(test_data[0][0], featurizer))
test_dsets.append(data.AtomDataset(test_data[0][1], featurizer))
test_dsets.append(data.BondDataset(test_data[0][2], featurizer))
test_dset = data.MolAtomBondDataset(test_dsets[0], test_dsets[1], test_dsets[2])

all_dsets = []
all_dsets.append(data.MoleculeDataset(all_data[0], featurizer))
all_atom_data = data.AtomDataset(all_data[1], featurizer)
all_dsets.append(all_atom_data)
atom_slices = all_atom_data._slices
all_bond_data = data.BondDataset(all_data[2], featurizer)
all_dsets.append(all_bond_data)
bond_slices = all_bond_data._slices
all_dset = data.MolAtomBondDataset(all_dsets[0], all_dsets[1], all_dsets[2])

## Get DataLoader

In [74]:
train_loader = data.build_dataloader(train_dset, num_workers=num_workers, shuffle=False)
val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)
test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)
all_loader = data.build_dataloader(all_dset, num_workers=num_workers, shuffle=False)

# Change Message-Passing Neural Network (MPNN) inputs here

## Message Passing
A `Message passing` constructs molecular graphs using message passing to learn node-level hidden representations.

Options are `mp = nn.BondMessagePassing()` or `mp = nn.AtomMessagePassing()`

In [75]:
mp = nn.MixedBondMessagePassing() #include why aggregation isn't used
mp

MixedBondMessagePassing(
  (W_i): Linear(in_features=86, out_features=300, bias=False)
  (W_h): Linear(in_features=300, out_features=300, bias=False)
  (W_o): Linear(in_features=372, out_features=300, bias=True)
  (W_o_b): Linear(in_features=314, out_features=300, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (tau): ReLU()
  (V_d_transform): Identity()
  (E_d_transform): Identity()
  (graph_transform): Identity()
)

## Feed-Forward Network (FFN)

A `FFN` takes the aggregated representations and make target predictions.

Available options can be found in `nn.PredictorRegistry`.

For regression:
- `ffn = nn.RegressionFFN()`
- `ffn = nn.MveFFN()`
- `ffn = nn.EvidentialFFN()`

For classification:
- `ffn = nn.BinaryClassificationFFN()`
- `ffn = nn.BinaryDirichletFFN()`
- `ffn = nn.MulticlassClassificationFFN()`
- `ffn = nn.MulticlassDirichletFFN()`

For spectral:
- `ffn = nn.SpectralFFN()` # will be available in future version

In [76]:
print(nn.PredictorRegistry)

ClassRegistry {
    'regression': <class 'chemprop.nn.predictors.RegressionFFN'>,
    'regression-mve': <class 'chemprop.nn.predictors.MveFFN'>,
    'regression-evidential': <class 'chemprop.nn.predictors.EvidentialFFN'>,
    'regression-quantile': <class 'chemprop.nn.predictors.QuantileFFN'>,
    'classification': <class 'chemprop.nn.predictors.BinaryClassificationFFN'>,
    'classification-dirichlet': <class 'chemprop.nn.predictors.BinaryDirichletFFN'>,
    'multiclass': <class 'chemprop.nn.predictors.MulticlassClassificationFFN'>,
    'multiclass-dirichlet': <class 'chemprop.nn.predictors.MulticlassDirichletFFN'>,
    'spectral': <class 'chemprop.nn.predictors.SpectralFFN'>
}


In [77]:
mol_output_transform = nn.UnscaleTransform.from_standard_scaler(mol_scaler)
atom_output_transform = nn.UnscaleTransform.from_standard_scaler(atom_scaler)
bond_output_transform = nn.UnscaleTransform.from_standard_scaler(bond_scaler)

In [78]:
mol_ffn = nn.RegressionFFN(output_transform=mol_output_transform, n_tasks=1)
atom_ffn = nn.RegressionFFN(output_transform=atom_output_transform, n_tasks=1)
bond_ffn = nn.RegressionFFN(output_transform=bond_output_transform, input_dim=600, n_tasks=1)

## Batch Norm
A `Batch Norm` normalizes the outputs of the aggregation by re-centering and re-scaling.

Whether to use batch norm

In [79]:
batch_norm = True

## Metrics
`Metrics` are the ways to evaluate the performance of model predictions.

Available options can be found in `metrics.MetricRegistry`, including

In [80]:
print(nn.metrics.MetricRegistry)

ClassRegistry {
    'mse': <class 'chemprop.nn.metrics.MSE'>,
    'mae': <class 'chemprop.nn.metrics.MAE'>,
    'rmse': <class 'chemprop.nn.metrics.RMSE'>,
    'bounded-mse': <class 'chemprop.nn.metrics.BoundedMSE'>,
    'bounded-mae': <class 'chemprop.nn.metrics.BoundedMAE'>,
    'bounded-rmse': <class 'chemprop.nn.metrics.BoundedRMSE'>,
    'r2': <class 'chemprop.nn.metrics.R2Score'>,
    'binary-mcc': <class 'chemprop.nn.metrics.BinaryMCCMetric'>,
    'multiclass-mcc': <class 'chemprop.nn.metrics.MulticlassMCCMetric'>,
    'roc': <class 'chemprop.nn.metrics.BinaryAUROC'>,
    'prc': <class 'chemprop.nn.metrics.BinaryAUPRC'>,
    'accuracy': <class 'chemprop.nn.metrics.BinaryAccuracy'>,
    'f1': <class 'chemprop.nn.metrics.BinaryF1Score'>
}


In [81]:
metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()] # Only the first metric is used for training and early stopping

## Constructs MPNN

In [82]:
agg = nn.MeanAggregation()
mol_atom_bond_mpnn = models.MolAtomBondMPNN(mp, agg, mol_ffn, atom_ffn, bond_ffn, batch_norm, metric_list)

mol_atom_bond_mpnn

/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'mol_predictor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['mol_predictor'])`.
/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'atom_predictor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['atom_predictor'])`.
/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'bond_predictor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['bond_predictor'])`.


MolAtomBondMPNN(
  (message_passing): MixedBondMessagePassing(
    (W_i): Linear(in_features=86, out_features=300, bias=False)
    (W_h): Linear(in_features=300, out_features=300, bias=False)
    (W_o): Linear(in_features=372, out_features=300, bias=True)
    (W_o_b): Linear(in_features=314, out_features=300, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (E_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): MeanAggregation()
  (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictors): ModuleList(
    (0-1): 2 x RegressionFFN(
      (ffn): MLP(
        (0): Sequential(
          (0): Linear(in_features=300, out_features=300, bias=True)
        )
        (1): Sequential(
          (0): ReLU()
          (1): Dropout(p=0.0, inplace=False)
          (2): Linear(in_features=300, out_features=1, bias=True)
        )
      )
      (criterion): MSE(task_weights=[[1.0]

# Set up trainer

In [83]:
# Configure model checkpointing
checkpointing = ModelCheckpoint(
    "checkpoints",  # Directory where model checkpoints will be saved
    "best-{epoch}-{val_loss:.2f}",  # Filename format for checkpoints, including epoch and validation loss
    "val_loss",  # Metric used to select the best checkpoint (based on validation loss)
    mode="min",  # Save the checkpoint with the lowest validation loss (minimization objective)
    save_last=True,  # Always save the most recent checkpoint, even if it's not the best
)

trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="cpu",
    devices=1,
    max_epochs=20, # number of epochs to train for
    callbacks=[checkpointing],
)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


# Start training

In [84]:
trainer.fit(mol_atom_bond_mpnn, train_loader, val_loader)

/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/brianli/Documents/chemprop/examples/checkpoints exists and is not empty.
Loading `train_dataloader` to estimate number of stepping batches.
/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.

  | Name            | Type                    | Params | Mode 
--------------------------------------------------------------------
0 | message_passing | MixedBondMessagePassing | 322 K  | train
1 | agg             | MeanAggregation         | 0      | train
2 | bn              | BatchNorm1d             | 600    | train
3 | predictors      | ModuleList              | 361 K  | train
4 | X_d_transform

/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 0: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 49.48it/s]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|████████████████████| 1/1 [00:00<00:00, 66.49it/s, val_loss=4.900]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|████████████████████| 1/1 [00:00<00:00, 91.30it/s, val_loss=4.800]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|████████████████████| 1/1 [00:00<00:00, 57.29it/s, val_loss=4.630]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|████████████████████| 1/1 [00:00<00:00, 88.28it/s, val_loss=4.470]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|████████████████████| 1/1 [00:00<00:00, 67.45it/s, val_loss=4.310]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 6: 100%|██████████████

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|███████████████████| 1/1 [00:00<00:00, 24.43it/s, val_loss=2.070]


# Test results

In [85]:
results = trainer.test(mol_atom_bond_mpnn, test_loader)

/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████████████████| 1/1 [00:00<00:00, 207.81it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     atom_mol_test/mae     │    1.2611865997314453     │
│    atom_mol_test/rmse     │     1.747967004776001     │
│  bond_atom_mol_test/mae   │    1.2611865997314453     │
│  bond_atom_mol_test/rmse  │     1.747967004776001     │
│       mol_test/mae        │    1.2611865997314453     │
│       mol_test/rmse       │     1.747967004776001     │
└───────────────────────────┴───────────────────────────┘

# Predictions

In [86]:
from chemprop.models import load_mixed_model
import torch

mol_individual_preds = []
atom_individual_preds = []
bond_individual_preds = []
model = load_mixed_model(checkpointing.best_model_path)
print(model.state_dict().keys())
trainer = pl.Trainer(
    logger=False,
    enable_progress_bar=True,
    accelerator="cpu",
    devices=1,
)

odict_keys(['message_passing.W_i.weight', 'message_passing.W_h.weight', 'message_passing.W_o.weight', 'message_passing.W_o.bias', 'message_passing.W_o_b.weight', 'message_passing.W_o_b.bias', 'bn.weight', 'bn.bias', 'bn.running_mean', 'bn.running_var', 'bn.num_batches_tracked', 'predictors.0.ffn.0.0.weight', 'predictors.0.ffn.0.0.bias', 'predictors.0.ffn.1.2.weight', 'predictors.0.ffn.1.2.bias', 'predictors.0.criterion.task_weights', 'predictors.0.output_transform.mean', 'predictors.0.output_transform.scale', 'predictors.1.ffn.0.0.weight', 'predictors.1.ffn.0.0.bias', 'predictors.1.ffn.1.2.weight', 'predictors.1.ffn.1.2.bias', 'predictors.1.criterion.task_weights', 'predictors.1.output_transform.mean', 'predictors.1.output_transform.scale', 'predictors.2.ffn.0.0.weight', 'predictors.2.ffn.0.0.bias', 'predictors.2.ffn.1.2.weight', 'predictors.2.ffn.1.2.bias', 'predictors.2.criterion.task_weights', 'predictors.2.output_transform.mean', 'predictors.2.output_transform.scale', 'metrics.0.0.

/Users/brianli/Documents/chemprop/chemprop/models/model.py:515: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  d = torch.load(path, map_location)
GPU available: True (mps), u

In [87]:
predss = trainer.predict(model, all_loader)
print(predss[0])
mol_individual_preds.append(torch.concat([predss[0][0]], 0))
atom_individual_preds.append(torch.concat([predss[0][1]], 0))
bond_individual_preds.append(torch.concat([predss[0][2]], 0))

mol_average_preds = torch.mean(torch.stack(mol_individual_preds).float(), dim=0)
atom_average_preds = torch.mean(torch.stack(atom_individual_preds).float(), dim=0)
bond_average_preds = torch.mean(torch.stack(bond_individual_preds).float(), dim=0)

test_path = chemprop_dir / "tests" / "data" / "mixed_regression_input.csv"
df_test = pd.read_csv(test_path, header="infer", index_col=False)

/opt/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|███████████████████| 1/1 [00:00<00:00, 170.39it/s]
[tensor([[4.3620],
        [4.0493],
        [7.4568],
        [7.3635],
        [6.2358],
        [7.6309],
        [6.9027],
        [7.3882],
        [7.4814],
        [8.6928]]), tensor([[2.5828],
        [2.5828],
        [2.5190],
        [2.5736],
        [2.5190],
        [2.5241],
        [2.4984],
        [2.6670],
        [3.3785],
        [2.5216],
        [2.6069],
        [3.4242],
        [3.4629],
        [2.5660],
        [3.5201],
        [3.5036],
        [2.5182],
        [2.6313],
        [3.2963],
        [3.5011],
        [3.4688],
        [2.5928],
        [3.3984],
        [3.5433],
        [3.4630],
        [2.7064],
        [3.4430],
        [2.5004],
        [2.7566],
        [3.3822],
        [3.6297],
        [3.6297]]), tensor([[1.3667],
        [1.3667],
        [1.5166],
        [1.5059],
        [1.5059],
        [1.5166],
        [1.5381],
        [1.5317],
        [1.683

## Loaded Model

In [88]:
model

MolAtomBondMPNN(
  (message_passing): MixedBondMessagePassing(
    (W_i): Linear(in_features=86, out_features=300, bias=False)
    (W_h): Linear(in_features=300, out_features=300, bias=False)
    (W_o): Linear(in_features=372, out_features=300, bias=True)
    (W_o_b): Linear(in_features=314, out_features=300, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (E_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): MeanAggregation()
  (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictors): ModuleList(
    (0-1): 2 x RegressionFFN(
      (ffn): MLP(
        (0): Sequential(
          (0): Linear(in_features=300, out_features=300, bias=True)
        )
        (1): Sequential(
          (0): ReLU()
          (1): Dropout(p=0.0, inplace=False)
          (2): Linear(in_features=300, out_features=1, bias=True)
        )
      )
      (criterion): MSE(task_weights=[[1.0]

In [89]:
target_cols = df_test.columns.tolist()
mol_cols, atom_cols, bond_cols = [], [], []
for i in range(1, len(target_cols)):
    index = 0
    column_type = df_test.iloc[index][target_cols[i]]
    if isinstance(column_type, float):
        mol_cols.append(i)
    else:
        column_mol = make_mol(df_test.iloc[index][target_cols[0]], False, False)
        column_type = ast.literal_eval(column_type)
        while index < len(df_test) and column_mol.GetNumAtoms() == column_mol.GetNumBonds():
            index += 1
            column_mol = make_mol(df_test.iloc[index][target_cols[0]])
        column_type = ast.literal_eval(df_test.iloc[index][target_cols[i]])
        atom_cols.append(i) if len(column_type) == column_mol.GetNumAtoms() else bond_cols.append(i)

In [91]:
print(atom_cols)
print(mol_average_preds)
df_test.iloc[:, mol_cols] = mol_average_preds.tolist()

for i in range(len(df_test)):
    first_atom = atom_slices.index(i)
    last_atom = first_atom + atom_slices.count(i)
    atom_preds = atom_average_preds[first_atom:last_atom]
    df_test.iloc[i, atom_cols] = [str(atom_preds[:,j].tolist()) for j in range(len(atom_cols))]

    first_bond = bond_slices.index(i)
    last_bond = first_bond + bond_slices.count(i)
    bond_preds = bond_average_preds[first_atom:last_atom]
    df_test.iloc[i, bond_cols] = [str(bond_preds[:,j].tolist()) for j in range(len(bond_cols))]

#torch.split(average_preds, split_size_or_sections=torch.bincount(torch.tensor(slices)).tolist(), dim=0)

output_path = chemprop_dir / "tests" / "data" / "mixed_regression_output.csv"
if output_path.suffix == ".pkl":
    df_test = df_test.reset_index(drop=True)
    df_test.to_pickle(output_path)
else:
    df_test.to_csv(output_path, index=False)

df_test

# do the reordering of target columns outside here in jupyter notebook but inside parsing.py in the cli

[2]
tensor([[4.3620],
        [4.0493],
        [7.4568],
        [7.3635],
        [6.2358],
        [7.6309],
        [6.9027],
        [7.3882],
        [7.4814],
        [8.6928]])


,smiles,molecule,atom,bond
0,CC,4.362026,"[2.582777738571167, 2.582777738571167]","[1.3666716814041138, 1.3666716814041138]"
1,CCC,4.049263,"[2.5190482139587402, 2.573632001876831, 2.5190...","[1.5166375637054443, 1.5058560371398926, 1.505..."
2,CCCO,7.456800,"[2.5240964889526367, 2.498371124267578, 2.6669...","[1.5166375637054443, 1.5381231307983398, 1.531..."
3,CCOO,7.363490,"[2.5215775966644287, 2.6069016456604004, 3.424...","[1.6841650009155273, 2.479125499725342, 2.4594..."
4,COO,6.235751,"[2.5659852027893066, 3.5201239585876465, 3.503...","[1.538448452949524, 2.192457914352417, 2.17960..."
5,CCOOO,7.630944,"[2.5181939601898193, 2.6313321590423584, 3.296...","[2.984977960586548, 2.9720447063446045, 1.9484..."
6,COOO,6.902697,"[2.592841148376465, 3.398362159729004, 3.54325...","[2.9119653701782227, 1.5535534620285034, 1.542..."
7,CO,7.388216,"[2.7064247131347656, 3.4429869651794434]","[2.3094632625579834, 2.811837673187256]"
8,CCO,7.481431,"[2.500397205352783, 2.7566416263580322, 3.3821...","[2.804741144180298, 3.0038018226623535, 2.9953..."
9,OO,8.692772,"[3.6297266483306885, 3.6297266483306885]","[2.0775113105773926, 2.058058023452759]"
